In [1]:
import os,re
from pyspark.sql.functions import col, array_contains, array_position,year,to_date

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1654087977069_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1654087977069_0002,pyspark,busy,Link,Link,
2,application_1654087977069_0003,pyspark,idle,Link,Link,✔


In [3]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1654087977069_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1654087977069_0002,pyspark,busy,Link,Link,
3,application_1654087977069_0004,pyspark,idle,Link,Link,✔


#### Installing Packages

In [4]:
#sc.uninstall_package('pandas')
sc.install_pypi_package("time")
sc.install_pypi_package("cython")
sc.install_pypi_package("boto3")
sc.install_pypi_package("s3fs")
sc.install_pypi_package("pandas") #Install pandas version 0.25.1 
sc.install_pypi_package("matplotlib==3.1.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


  Using cached https://files.pythonhosted.org/packages/a5/a8/a8480dc84e4677f966839035a66f2fd3a8a63e44f246571d77afa9045769/Cython-0.29.30-cp27-cp27mu-manylinux_2_5_x86_64.manylinux1_x86_64.whl

  Found existing installation: botocore 1.18.6
    Not uninstalling botocore at /usr/lib/python2.7/site-packages, outside environment /tmp/1654088371957-0
  Found existing installation: s3transfer 0.3.3
    Not uninstalling s3transfer at /usr/lib/python2.7/site-packages, outside environment /tmp/1654088371957-0

  Running setup.py bdist_wheel for s3fs: started
  Running setup.py bdist_wheel for s3fs: finished with status 'done'
  Stored in directory: /var/lib/livy/.cache/pip/wheels/94/20/c9/b0256fe423de22bca83518fe0f7eba34ae1b9591a9d8c2d3b4
Successfully built s3fs
  Found existing installation: six 1.9.0
    Not uninstalling six at /usr/lib/python2.7/site-packages, outside environment /tmp/1654088371957-0

  Using cached https://files.pythonhosted.org/packages/db/83/7d4008ffc2988066ff37f6a0bb6d7

In [5]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

amazon-linux-extras (2.0.1)
awscli (1.18.147)
Babel (0.9.6)
backports.ssl-match-hostname (3.5.0.1)
beautifulsoup4 (4.9.3)
boto3 (1.17.112)
botocore (1.20.112)
cffi (1.6.0)
chardet (2.2.1)
click (7.1.2)
cloud-init (19.3)
colorama (0.3.2)
configobj (4.7.2)
cryptography (1.7.2)
Cython (0.29.30)
docutils (0.12)
ec2-hibinit-agent (1.0.2)
enum34 (1.0.4)
futures (3.0.5)
hibagent (1.1.0)
idna (2.4)
iniparse (0.4)
ipaddress (1.0.16)
javapackages (1.0.0)
Jinja2 (2.7.2)
jmespath (0.9.3)
joblib (0.14.1)
jsonpatch (1.2)
jsonpointer (1.9)
jsonschema (2.5.1)
jwcrypto (0.4.2)
kitchen (1.1.1)
langtable (0.0.31)
lockfile (0.9.1)
lxml (3.2.1)
MarkupSafe (0.11)
mysqlclient (1.4.2)
nltk (3.6.5)
nose (1.3.4)
numpy (1.16.5)
oauthlib (2.0.1)
pandas (0.24.2)
Pillow (2.0.0)
pip (9.0.1)
ply (3.4)
pyasn1 (0.1.9)
pycparser (2.14)
pycurl (7.19.0)
pygpgme (0.3)
pyliblzma (0.5.3)
pyparsing (1.5.6)
pystache (0.5.3)
python-daemon (1.6)
python-dateutil (2.6.0)
python27-sagemaker-pyspark (1.4.1)
pytz (2022.1)
pyxattr (0.

#### Fetch Data

In [6]:
df_2019=spark.read.parquet("s3://pas-biogen-sagemaker/Antony/Ongoing_Output/2019_historical/part-00000-aa403f3a-0fe6-4c9b-a1f2-9b83b9f27ae8-c000.snappy.parquet")
df_2020=spark.read.parquet("s3://pas-biogen-sagemaker/Antony/Ongoing_Output/2020_Historical/part-00000-8d4e6c0f-e97d-4dcd-8b91-994ef76de8c9-c000.snappy.parquet")
df_2021=spark.read.parquet("s3://pas-biogen-sagemaker/Antony/Ongoing_Output/2021_historical/part-00000-2a96728d-965e-40ed-8c27-1f828c7d9e77-c000.snappy.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Count of Patients

In [7]:
print("No of unique patients in 2019 - ",df_2019[['patient_key']].distinct().count())
print("No of unique patients in 2020 - ",df_2020[['patient_key']].distinct().count())
print("No of unique patients in 2021 - ",df_2021[['patient_key']].distinct().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('No of unique patients in 2019 - ', 92928)
('No of unique patients in 2020 - ', 91360)
('No of unique patients in 2021 - ', 60500)

In [8]:
df_2019.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['patient_key', 'UMLS_CUI', 'provider_id', 'facility_id', 'text', 'label', 'negex', 'span', 'UMLS_Name', 'TUIs', 'Child_semantic_label', 'Parent_semantic_label', 'Similarity_Score', 'Thought_Disturbance', 'Abnormal_Behavior', 'Suicidal_Intent', 'Disorientated', 'Mental_Health_Impairment', 'Memory_Loss', 'Clouded_Consciousness', 'Agitation', 'Congnitive_Impairment', 'Alzheimer', 'pat_last_name', 'pat_first_name', 'pat_mrn', 'pat_dob', 'pat_gender', 'pat_race', 'pat_ethinicity', 'pat_city', 'pat_zip', 'pat_county', 'pat_state', 'pat_country', 'pat_marital_status', 'pat_smoking_status', 'pat_status']

#### Calculating Age

In [9]:
from pyspark.sql.functions import substring, col
#Result
df_2019 = df_2019.withColumn("year", substring(col("pat_dob"), 7, 10))
df_2019 = df_2019.withColumn("age", (2022 - df_2019['year']))
df_2019 = df_2019.withColumn("pat_age",df_2019.age.cast('int'))

df_2020 = df_2020.withColumn("year", substring(col("pat_dob"), 7, 10))
df_2020 = df_2020.withColumn("age", (2022 - df_2020['year']))
df_2020 = df_2020.withColumn("pat_age",df_2020.age.cast('int'))

df_2021 = df_2021.withColumn("year", substring(col("pat_dob"), 7, 10))
df_2021 = df_2021.withColumn("age", (2022 - df_2021['year']))
df_2021 = df_2021.withColumn("pat_age",df_2021.age.cast('int'))

df_2019[['pat_dob','pat_age']].show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+
|   pat_dob|pat_age|
+----------+-------+
|05-05-1966|     56|
|05-05-1966|     56|
|05-05-1966|     56|
|05-05-1966|     56|
|05-05-1966|     56|
+----------+-------+
only showing top 5 rows

In [10]:
from pyspark.sql.functions import *
df_2019 = df_2019.withColumn('Thought_Disturbance', regexp_replace('Thought_Disturbance', 'false', '0'))
df_2019 = df_2019.withColumn('Thought_Disturbance', regexp_replace('Thought_Disturbance', 'true', '1'))
df_2019 = df_2019.withColumn('Abnormal_Behavior', regexp_replace('Abnormal_Behavior', 'false', '0'))
df_2019 = df_2019.withColumn('Abnormal_Behavior', regexp_replace('Abnormal_Behavior', 'true', '1'))
df_2019 = df_2019.withColumn('Suicidal_Intent', regexp_replace('Suicidal_Intent', 'false', '0'))
df_2019 = df_2019.withColumn('Suicidal_Intent', regexp_replace('Suicidal_Intent', 'true', '1'))
df_2019 = df_2019.withColumn('Disorientated', regexp_replace('Disorientated', 'false', '0'))
df_2019 = df_2019.withColumn('Disorientated', regexp_replace('Disorientated', 'true', '1'))
df_2019 = df_2019.withColumn('Mental_Health_Impairment', regexp_replace('Mental_Health_Impairment', 'false', '0'))
df_2019 = df_2019.withColumn('Mental_Health_Impairment', regexp_replace('Mental_Health_Impairment', 'true', '1'))
df_2019 = df_2019.withColumn('Memory_Loss', regexp_replace('Memory_Loss', 'false', '0'))
df_2019 = df_2019.withColumn('Memory_Loss', regexp_replace('Memory_Loss', 'true', '1'))
df_2019 = df_2019.withColumn('Clouded_Consciousness', regexp_replace('Clouded_Consciousness', 'false', '0'))
df_2019 = df_2019.withColumn('Clouded_Consciousness', regexp_replace('Clouded_Consciousness', 'true', '1'))
df_2019 = df_2019.withColumn('Agitation', regexp_replace('Agitation', 'false', '0'))
df_2019 = df_2019.withColumn('Agitation', regexp_replace('Agitation', 'true', '1'))
df_2019 = df_2019.withColumn('Congnitive_Impairment', regexp_replace('Congnitive_Impairment', 'false', '0'))
df_2019 = df_2019.withColumn('Congnitive_Impairment', regexp_replace('Congnitive_Impairment', 'true', '1'))
df_2019 = df_2019.withColumn('Alzheimer', regexp_replace('Alzheimer', 'false', '0'))
df_2019 = df_2019.withColumn('Alzheimer', regexp_replace('Alzheimer', 'true', '1'))
df_2020 = df_2020.withColumn('Thought_Disturbance', regexp_replace('Thought_Disturbance', 'false', '0'))
df_2020 = df_2020.withColumn('Thought_Disturbance', regexp_replace('Thought_Disturbance', 'true', '1'))
df_2020 = df_2020.withColumn('Abnormal_Behavior', regexp_replace('Abnormal_Behavior', 'false', '0'))
df_2020 = df_2020.withColumn('Abnormal_Behavior', regexp_replace('Abnormal_Behavior', 'true', '1'))
df_2020 = df_2020.withColumn('Suicidal_Intent', regexp_replace('Suicidal_Intent', 'false', '0'))
df_2020 = df_2020.withColumn('Suicidal_Intent', regexp_replace('Suicidal_Intent', 'true', '1'))
df_2020 = df_2020.withColumn('Disorientated', regexp_replace('Disorientated', 'false', '0'))
df_2020 = df_2020.withColumn('Disorientated', regexp_replace('Disorientated', 'true', '1'))
df_2020 = df_2020.withColumn('Mental_Health_Impairment', regexp_replace('Mental_Health_Impairment', 'false', '0'))
df_2020 = df_2020.withColumn('Mental_Health_Impairment', regexp_replace('Mental_Health_Impairment', 'true', '1'))
df_2020 = df_2020.withColumn('Memory_Loss', regexp_replace('Memory_Loss', 'false', '0'))
df_2020 = df_2020.withColumn('Memory_Loss', regexp_replace('Memory_Loss', 'true', '1'))
df_2020 = df_2020.withColumn('Clouded_Consciousness', regexp_replace('Clouded_Consciousness', 'false', '0'))
df_2020 = df_2020.withColumn('Clouded_Consciousness', regexp_replace('Clouded_Consciousness', 'true', '1'))
df_2020 = df_2020.withColumn('Agitation', regexp_replace('Agitation', 'false', '0'))
df_2020 = df_2020.withColumn('Agitation', regexp_replace('Agitation', 'true', '1'))
df_2020 = df_2020.withColumn('Congnitive_Impairment', regexp_replace('Congnitive_Impairment', 'false', '0'))
df_2020 = df_2020.withColumn('Congnitive_Impairment', regexp_replace('Congnitive_Impairment', 'true', '1'))
df_2020 = df_2020.withColumn('Alzheimer', regexp_replace('Alzheimer', 'false', '0'))
df_2020 = df_2020.withColumn('Alzheimer', regexp_replace('Alzheimer', 'true', '1'))
df_2021 = df_2021.withColumn('Thought_Disturbance', regexp_replace('Thought_Disturbance', 'false', '0'))
df_2021 = df_2021.withColumn('Thought_Disturbance', regexp_replace('Thought_Disturbance', 'true', '1'))
df_2021 = df_2021.withColumn('Abnormal_Behavior', regexp_replace('Abnormal_Behavior', 'false', '0'))
df_2021 = df_2021.withColumn('Abnormal_Behavior', regexp_replace('Abnormal_Behavior', 'true', '1'))
df_2021 = df_2021.withColumn('Suicidal_Intent', regexp_replace('Suicidal_Intent', 'false', '0'))
df_2021 = df_2021.withColumn('Suicidal_Intent', regexp_replace('Suicidal_Intent', 'true', '1'))
df_2021 = df_2021.withColumn('Disorientated', regexp_replace('Disorientated', 'false', '0'))
df_2021 = df_2021.withColumn('Disorientated', regexp_replace('Disorientated', 'true', '1'))
df_2021 = df_2021.withColumn('Mental_Health_Impairment', regexp_replace('Mental_Health_Impairment', 'false', '0'))
df_2021 = df_2021.withColumn('Mental_Health_Impairment', regexp_replace('Mental_Health_Impairment', 'true', '1'))
df_2021 = df_2021.withColumn('Memory_Loss', regexp_replace('Memory_Loss', 'false', '0'))
df_2021 = df_2021.withColumn('Memory_Loss', regexp_replace('Memory_Loss', 'true', '1'))
df_2021 = df_2021.withColumn('Clouded_Consciousness', regexp_replace('Clouded_Consciousness', 'false', '0'))
df_2021 = df_2021.withColumn('Clouded_Consciousness', regexp_replace('Clouded_Consciousness', 'true', '1'))
df_2021 = df_2021.withColumn('Agitation', regexp_replace('Agitation', 'false', '0'))
df_2021 = df_2021.withColumn('Agitation', regexp_replace('Agitation', 'true', '1'))
df_2021 = df_2021.withColumn('Congnitive_Impairment', regexp_replace('Congnitive_Impairment', 'false', '0'))
df_2021 = df_2021.withColumn('Congnitive_Impairment', regexp_replace('Congnitive_Impairment', 'true', '1'))
df_2021 = df_2021.withColumn('Alzheimer', regexp_replace('Alzheimer', 'false', '0'))
df_2021 = df_2021.withColumn('Alzheimer', regexp_replace('Alzheimer', 'true', '1'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Aggregating the outputs

In [11]:
from pyspark.sql import functions as F

# Overall Data
df_ov_2019 = df_2019.groupBy(df_2019['patient_key']).agg(F.first(df_2019['pat_first_name']), F.first(df_2019['pat_last_name']),F.first(df_2019['pat_mrn']),F.first(df_2019['pat_gender']),F.first(df_2019['pat_race']),F.first(df_2019['pat_ethinicity']),F.first(df_2019['pat_city']),F.first(df_2019['pat_zip']),F.first(df_2019['pat_county']),F.first(df_2019['pat_state']),F.first(df_2019['pat_marital_status']),F.first(df_2019['pat_smoking_status']),F.first(df_2019['pat_status']),
                                                   F.first(df_2019['pat_dob']),F.max(df_2019['pat_age']),F.max(df_2019['Thought_Disturbance']), F.max(df_2019['Abnormal_Behavior']), F.max(df_2019['Suicidal_Intent']), F.max(df_2019['Disorientated']), F.max(df_2019['Mental_Health_Impairment']), F.max(df_2019['Memory_Loss']), F.max(df_2019['Clouded_Consciousness']), F.max(df_2019['Agitation']), F.max(df_2019['Congnitive_Impairment']),F.max(df_2019['Alzheimer']))


df_ov_2020 = df_2020.groupBy(df_2020['patient_key']).agg(F.first(df_2020['pat_first_name']), F.first(df_2020['pat_last_name']),F.first(df_2020['pat_mrn']),F.first(df_2020['pat_gender']),F.first(df_2020['pat_race']),F.first(df_2020['pat_ethinicity']),F.first(df_2020['pat_city']),F.first(df_2020['pat_zip']),F.first(df_2020['pat_county']),F.first(df_2020['pat_state']),F.first(df_2020['pat_marital_status']),F.first(df_2020['pat_smoking_status']),F.first(df_2020['pat_status']),
                                                   F.max(df_2020['pat_dob']),F.max(df_2020['pat_age']),F.max(df_2020['Thought_Disturbance']), F.max(df_2020['Abnormal_Behavior']), F.max(df_2020['Suicidal_Intent']), F.max(df_2020['Disorientated']), F.max(df_2020['Mental_Health_Impairment']), F.max(df_2020['Memory_Loss']), F.max(df_2020['Clouded_Consciousness']), F.max(df_2020['Agitation']), F.max(df_2020['Congnitive_Impairment']),F.max(df_2020['Alzheimer']))



df_ov_2021 = df_2021.groupBy(df_2021['patient_key']).agg(F.first(df_2021['pat_first_name']), F.first(df_2021['pat_last_name']),F.first(df_2021['pat_mrn']),F.first(df_2021['pat_gender']),F.first(df_2021['pat_race']),F.first(df_2021['pat_ethinicity']),F.first(df_2021['pat_city']),F.first(df_2021['pat_zip']),F.first(df_2021['pat_county']),F.first(df_2021['pat_state']),F.first(df_2021['pat_marital_status']),F.first(df_2021['pat_smoking_status']),F.first(df_2021['pat_status']),
                                                   F.max(df_2021['pat_dob']),F.max(df_2021['pat_age']),F.max(df_2021['Thought_Disturbance']), F.max(df_2021['Abnormal_Behavior']), F.max(df_2021['Suicidal_Intent']), F.max(df_2021['Disorientated']), F.max(df_2021['Mental_Health_Impairment']), F.max(df_2021['Memory_Loss']), F.max(df_2021['Clouded_Consciousness']), F.max(df_2021['Agitation']), F.max(df_2021['Congnitive_Impairment']),F.max(df_2021['Alzheimer']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#Rename Columns
overall_names = ['patient_key', 'pat_first_name', 'pat_last_name', 'pat_mrn', 'pat_gender', 'pat_race', 'pat_ethinicity', 'pat_city', 'pat_zip', 'pat_county', 'pat_state', 'pat_marital_status', 'pat_smoking_status', 'pat_status', 'pat_dob','pat_age', 'Thought_Disturbance', 'Abnormal_Behavior', 'Suicidal_Intent', 'Disorientated', 'Mental_Health_Impairment', 'Memory_Loss', 'Clouded_Consciousness', 'Agitation', 'Congnitive_Impairment', 'Alzheimer']
df_ov_2019 = df_ov_2019.toDF(*overall_names)
df_ov_2020 = df_ov_2020.toDF(*overall_names)
df_ov_2021 = df_ov_2021.toDF(*overall_names)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_ov_2019.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

92928

In [14]:
# Function to merge Dataframe
from functools import reduce
from pyspark.sql import DataFrame
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#Merge Dataframe
df_overall = unionAll(df_ov_2019,df_ov_2020,df_ov_2021)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
print(df_overall.count())
print(df_overall.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

244788
['patient_key', 'pat_first_name', 'pat_last_name', 'pat_mrn', 'pat_gender', 'pat_race', 'pat_ethinicity', 'pat_city', 'pat_zip', 'pat_county', 'pat_state', 'pat_marital_status', 'pat_smoking_status', 'pat_status', 'pat_dob', 'pat_age', 'Thought_Disturbance', 'Abnormal_Behavior', 'Suicidal_Intent', 'Disorientated', 'Mental_Health_Impairment', 'Memory_Loss', 'Clouded_Consciousness', 'Agitation', 'Congnitive_Impairment', 'Alzheimer']

In [17]:
df_overall = df_overall.withColumn("pat_zip",df_overall.pat_zip.cast('int'))
df_overall = df_overall.withColumn('pat_gender', regexp_replace('pat_gender', 'Female', 'F'))
df_overall = df_overall.withColumn('pat_gender', regexp_replace('pat_gender', 'Male', 'M'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df_overall = df_overall.groupBy(df_overall['patient_key']).agg(F.first(df_overall['pat_first_name']), F.first(df_overall['pat_last_name']),F.first(df_overall['pat_mrn']),F.first(df_overall['pat_gender']),F.first(df_overall['pat_race']),F.first(df_overall['pat_ethinicity']),F.first(df_overall['pat_city']),F.first(df_overall['pat_zip']),F.first(df_overall['pat_county']),F.first(df_overall['pat_state']),F.first(df_overall['pat_marital_status']),F.first(df_overall['pat_smoking_status']),F.first(df_overall['pat_status']),
                                                   F.first(df_overall['pat_dob']),F.max(df_overall['pat_age']),F.max(df_overall['Thought_Disturbance']), F.max(df_overall['Abnormal_Behavior']), F.max(df_overall['Suicidal_Intent']), F.max(df_overall['Disorientated']), F.max(df_overall['Mental_Health_Impairment']), F.max(df_overall['Memory_Loss']), F.max(df_overall['Clouded_Consciousness']), F.max(df_overall['Agitation']), F.max(df_overall['Congnitive_Impairment']),F.max(df_overall['Alzheimer']))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
overall_names = ['patient_key', 'pat_first_name', 'pat_last_name', 'pat_mrn', 'pat_gender', 'pat_race', 'pat_ethinicity', 'pat_city', 'pat_zip', 'pat_county', 'pat_state', 'pat_marital_status', 'pat_smoking_status', 'pat_status', 'pat_dob','pat_age', 'Thought_Disturbance', 'Abnormal_Behavior', 'Suicidal_Intent', 'Disorientated', 'Mental_Health_Impairment', 'Memory_Loss', 'Clouded_Consciousness', 'Agitation', 'Congnitive_Impairment', 'Alzheimer']
df_overall = df_overall.toDF(*overall_names)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_overall.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

118671

In [21]:
#Patients having Alzheimer
#Result
df_alz = df_overall[df_overall['Alzheimer'] == 1].dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df_alz.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1676

In [23]:
#Saving CSV
df_overall.coalesce(1).write.format("csv").mode("append").save("s3://pas-biogen-sagemaker/Antony/Ongoing_Output/Alzheimer_Historical",header = 'true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
print("Total number of patients: ", df_overall[['patient_key']].distinct().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Total number of patients: ', 118671)

In [24]:
print("Total number of patients having Alzheimer: ", df_alz[['patient_key']].distinct().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Total number of patients having Alzheimer: ', 1676)

In [26]:
#Patients having Anchor features Overall vs ALZ
### Thought_Disturbance
all_Thought_Disturbance = df_overall[df_overall['Thought_Disturbance'] == 1].select('patient_key')
print('Overall Thought_Disturbance', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Thought_Disturbance'] == 1].select('patient_key')
print('Alzheimer Thought_Disturbance', alz_Thought_Disturbance[['patient_key']].distinct().count())

### Abnormal_Behavior
all_Thought_Disturbance = df_overall[df_overall['Abnormal_Behavior'] == 1].select('patient_key')
print('Overall Abnormal_Behavior', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Abnormal_Behavior'] == 1].select('patient_key')
print('Alzheimer Abnormal_Behavior', alz_Thought_Disturbance[['patient_key']].distinct().count())

### Suicidal_Intent
all_Thought_Disturbance = df_overall[df_overall['Suicidal_Intent'] == 1].select('patient_key')
print('Overall Suicidal_Intent', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Suicidal_Intent'] == 1].select('patient_key')
print('Alzheimer Suicidal_Intent', alz_Thought_Disturbance[['patient_key']].distinct().count())

### Disorientated
all_Thought_Disturbance = df_overall[df_overall['Disorientated'] == 1].select('patient_key')
print('Overall Disorientated', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Disorientated'] == 1].select('patient_key')
print('Alzheimer Disorientated', alz_Thought_Disturbance[['patient_key']].distinct().count())


### Mental_Health_Impairment
all_Mental_Health_Impairment = df_overall[df_overall['Mental_Health_Impairment'] == 1].select('patient_key')
print('Overall Mental_Health_Impairment', all_Mental_Health_Impairment[['patient_key']].distinct().count())
alz_Mental_Health_Impairment = df_alz[df_alz['Mental_Health_Impairment'] == 1].select('patient_key')
print('Alzheimer Mental_Health_Impairment', alz_Mental_Health_Impairment[['patient_key']].distinct().count())


### Memory_Loss
all_Thought_Disturbance = df_overall[df_overall['Memory_Loss'] == 1].select('patient_key')
print('Overall Memory_Loss', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Memory_Loss'] == 1].select('patient_key')
print('Alzheimer Memory_Loss', alz_Thought_Disturbance[['patient_key']].distinct().count())

### Clouded_Consciousness
all_Thought_Disturbance = df_overall[df_overall['Clouded_Consciousness'] == 1].select('patient_key')
print('Overall Clouded_Consciousness', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Clouded_Consciousness'] == 1].select('patient_key')
print('Alzheimer Clouded_Consciousness', alz_Thought_Disturbance[['patient_key']].distinct().count())

### Agitation
all_Thought_Disturbance = df_overall[df_overall['Agitation'] == 1].select('patient_key')
print('Overall Agitation', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Agitation'] == 1].select('patient_key')
print('Alzheimer Agitation', alz_Thought_Disturbance[['patient_key']].distinct().count())

### Congnitive_Impairment
all_Thought_Disturbance = df_overall[df_overall['Congnitive_Impairment'] == 1].select('patient_key')
print('Overall Congnitive_Impairment', all_Thought_Disturbance[['patient_key']].distinct().count())
alz_Thought_Disturbance = df_alz[df_alz['Congnitive_Impairment'] == 1].select('patient_key')
print('Alzheimer Congnitive_Impairment', alz_Thought_Disturbance[['patient_key']].distinct().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Overall Thought_Disturbance', 4040)
('Alzheimer Thought_Disturbance', 221)
('Overall Abnormal_Behavior', 17466)
('Alzheimer Abnormal_Behavior', 648)
('Overall Suicidal_Intent', 7792)
('Alzheimer Suicidal_Intent', 416)
('Overall Disorientated', 804)
('Alzheimer Disorientated', 93)
('Overall Mental_Health_Impairment', 35292)
('Alzheimer Mental_Health_Impairment', 1009)
('Overall Memory_Loss', 16207)
('Alzheimer Memory_Loss', 748)
('Overall Clouded_Consciousness', 7695)
('Alzheimer Clouded_Consciousness', 354)
('Overall Agitation', 6870)
('Alzheimer Agitation', 388)
('Overall Congnitive_Impairment', 45311)
('Alzheimer Congnitive_Impairment', 1256)

In [27]:
### Total Patients having Alzheimer
alz_cnt = df_alz[['patient_key']].distinct().count()
print('Alzheimer count', alz_cnt)

#Gender
all_Gender_M = df_overall[df_overall['pat_gender'] == 'M'].select('patient_key')
all_Gender_F = df_overall[df_overall['pat_gender'] == 'F'].select('patient_key')


alz_Gender_M = df_alz[df_alz['pat_gender'] == 'M'].select('patient_key')
alz_Gender_F = df_alz[df_alz['pat_gender'] == 'F'].select('patient_key')


print('all_Gender_M', all_Gender_M[['patient_key']].distinct().count())
print('all_Gender_F', all_Gender_F[['patient_key']].distinct().count())
print('alz_Gender_M', alz_Gender_M[['patient_key']].distinct().count())
print('alz_Gender_F', alz_Gender_F[['patient_key']].distinct().count())


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Alzheimer count', 1676)
('all_Gender_M', 55901)
('all_Gender_F', 62770)
('alz_Gender_M', 795)
('alz_Gender_F', 881)

In [28]:
from pyspark.sql.functions import when,col
conditions = when(col("pat_age") < 50, "<50").when(col("pat_age").between(50,55), "50-55").when(col("pat_age").between(55,60), "55-60").when(col("pat_age").between(60,65), "60-65").when(col("pat_age").between(65,70), "65-70").when(col("pat_age").between(70,75), "70-75").when(col("pat_age").between(75,80), "75-80").when(col("pat_age").between(80,85), "80-85").when(col("pat_age").between(85,90), "85-90").when(col("pat_age").between(90,150), ">90")
df_alz = df_alz.withColumn("age_catg", conditions)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df_age = df_alz.select(['patient_key','age_catg']).drop_duplicates()
df_age = df_age.groupBy('age_catg').count().orderBy('count', ascending=False).limit(10).toPandas()
df_age

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  age_catg  count
0    80-85    354
1    75-80    307
2    70-75    281
3    65-70    217
4    60-65    174
5    85-90    141
6    55-60    134
7    50-55     68

In [30]:
df_city = df_alz.select(['patient_key','pat_city']).drop_duplicates()
df_city = df_city.groupBy('pat_city').count().orderBy('count', ascending=False).limit(10).toPandas()
df_city

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

         pat_city  count
0       CLEVELAND    220
1           PARMA    123
2          ELYRIA     69
3          MENTOR     48
4     PAINESVILLE     35
5         CHARDON     31
6  NORTH ROYALTON     30
7        WESTLAKE     29
8          EUCLID     27
9       BEACHWOOD     25